# Digital Tools for Finance - Project
**Research Question:**\
What is the effect of the U.S. GDP growth rate on different S&P500 sector indices' market capitalizations\

**Authors:**\
Jonas Neller, 21-730-676\
Lorena Tassone, 18-700-237\
Naomi Huser, 17-056-201

## ToDo List:
API einrichten mit FRED\
Up- and downstream functions for database\
<s>Data cleaning</s> -> NaN values?\
Regressions\
plots

create docker container for jupyter-notebook and app

overleaf doc\
overleaf presentation

--> interactive app\
--> jupyter notebook \
--> good comments in code and github commits!


Notes:
- Fore presentation use beamer!
- Use ipywidget for app of Jupyter Notebook!

## Data Cleaning

In [53]:
# libraries
import numpy as np
import pandas as pd
import os
from functools import reduce


In [54]:
# create dataframes
data_mcap = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Market_Cap')
data_gdp = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Real_GDP').set_index('Date ')
data_unempr = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Unemployment_rate').set_index('Date ')
data_crudeprice = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Crude_price').set_index('Date ')
data_feddebt = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Federal_debt').set_index('Date ')
data_fedrate = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Federal_rate').set_index('Date ')

data_mcap = data_mcap.transpose()
data_mcap.columns = data_mcap.loc['Sector',:]


# create sector subsets
sectors = data_mcap.loc['Sector',:].unique()
print(sectors)

communication = data_mcap[['Communication Services']]  
consumerdisc = data_mcap[['Consumer Discretionary']] 
consumerstap = data_mcap[['Consumer Staples']] 
energy = data_mcap[['Energy']] 
financials = data_mcap[['Financials']] 
healthcare = data_mcap[['Health Care']] 
industrials = data_mcap[['Industrials']] 
informationtech = data_mcap[['Information Technology']] 
materials = data_mcap[['Materials']] 
realestate = data_mcap[['Real Estate']] 
utilities = data_mcap[['Utilities']] 

utilities

['Communication Services' 'Consumer Discretionary' 'Consumer Staples'
 'Energy' 'Financials' 'Health Care' 'Industrials'
 'Information Technology' 'Materials' 'Real Estate' 'Utilities']


Sector,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,...,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities
Name,AES CORP,ALLIANT ENERGY CORP,AMEREN CORPORATION,AMERICAN ELECTRIC POWER,AMERICAN WATER WORKS CO INC,ATMOS ENERGY CORP,CENTERPOINT ENERGY INC,CMS ENERGY CORP,CONSOLIDATED EDISON INC,CONSTELLATION ENERGY,...,NISOURCE INC,NRG ENERGY INC,P G & E CORP,PINNACLE WEST CAPITAL,PPL CORP,PUBLIC SERVICE ENTERPRISE GP,SEMPRA ENERGY,SOUTHERN CO/THE,WEC ENERGY GROUP INC,XCEL ENERGY INC
ISIN,US00130H1059,US0188021085,US0236081024,US0255371017,US0304201033,US0495601058,US15189T1079,US1258961002,US2091151041,US21037T1097,...,US65473P1057,US6293775085,US69331C1080,US7234841010,US69351T1060,US7445731067,US8168511090,US8425871071,US92939U1060,US98389B1008
Sector,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,...,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities,Utilities
2006-12-29 00:00:00,14638712578.440001,4384709584.45254,11082465319.5,16843471727.5,NaN,2783812955.56,5186872195.1,3714659289.6,12344091521.74,NaN,...,2585436457.09001,7082069181.42,16509186008.719999,5042522737.47,12668866168.236,16741258572.139999,14692202603.76,27377404804.080002,5551817787.18,9382708997.219999
2007-03-30 00:00:00,14293334604.719999,5203142483.98959,10374986145.0,19337893380.0,NaN,2770689248.16,5742217475.28,3973954379.2,13155459853.74,NaN,...,2630056060.93063,8813046970.639999,16933949864.25,4799797239.75,14457495295.370501,20942966433.119999,16040666104.379999,27435978163.0,5674427245.96,10067390534.67
2007-06-29 00:00:00,14619747943.040001,4510086514.57497,10170069510.9,17960461536.32,NaN,2669515424.1,5581703213.4,3840000860.8,12205768099.200001,NaN,...,2230398860.43326,10073112452.18,15922900532.4,3964184870.55,16710462345.6555,22263327147.599998,15625445214.120001,25779517271.189999,5172710574.79,8370499773.25
2007-09-28 00:00:00,13399013097.120001,4448559078.15501,10899085680.0,18395319997.439999,NaN,2525014003.68,5148532071.2,3777497632.16,12546154676.9,NaN,...,2062030361.8079,10142537781.450001,16890508337.0,3963216017.88,16457065022.064199,22374390646.650002,15345081970.639999,27443191550.279999,5266183705.17,9044078162.52
2007-12-31 00:00:00,14315413925.07,4723691869.76736,11276176509.389999,18643867338.240002,NaN,2516583130.2,5503085216.85,3903264497.44,13237141597.549999,NaN,...,2035175796.14,10343243513.459999,15256086158.67,4257329376.76,17988112199.596802,24983726704.959999,16176116207.16,29429803538.75,5696297037.12,9477821860.459999
2008-03-31 00:00:00,11189927437.98,3866971305.3726,9197420441.040001,16669763687.52,NaN,2288618752.5,4684342634.52,3048897541.34,10803922468.5,NaN,...,1857445107.65951,9218884263.26,13123393899.559999,3521507062.88,15877949067.0611,20441693328.02,13922387942.4,27231399981.990002,5143660652.47,8561502181.05
2008-06-30 00:00:00,12905493651.459999,3783440279.20662,8819415649.98,16156006131.15,3548800000.0,2486498185.65,5268654469.8,3355138357.9,10657320170.49,NaN,...,1930919693.56965,10121052813.299999,14179609590.93,3088847557.72,18091052073.992401,23361208622.939999,14131787158.6,26894940376.32,5287119732.02,8647303241.34


In [55]:
# merge factors by quarterly date
data_frames = [data_gdp, data_unempr, data_crudeprice, data_feddebt, data_fedrate]
data_factors_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date '], how='inner'), data_frames)
data_factors_merged

,Real_GDP,Unemployment_rate,Crude_price,Federal_debt,Federal_rate
Date,,,,,
2007-01-01,15478.956,4.6,54.780000,8849665,5.25
2007-04-01,15577.779,4.5,63.974762,8867677,5.25
2007-07-01,15671.605,4.7,74.046364,9007653,5.26
2007-10-01,15767.146,4.7,85.926522,9229172,4.76
2008-01-01,15702.906,5.0,92.982609,9437594,3.94
2008-04-01,15792.773,5.0,112.336364,9492006,2.28
2008-07-01,15709.562,5.8,133.958261,10024725,2.01
2008-10-01,15366.607,6.5,76.283913,10699805,0.97
2009-01-01,15187.475,7.8,41.630909,11126941,0.15


In [56]:
# checking timeframe length
print(len(utilities.index)-3)
print(len(data_factors_merged))

57
57


In [ ]:
# take average mcap of each sector


## Regression